## Word Embedding
---


In [1]:
# Import
from nltk.tokenize import word_tokenize
from gensim.models.phrases import Phrases, Phraser
#import fuzzysearch
#import fuzzywuzzy
import pandas as pd
import PyPDF2

pdfFileObj = open('20221118-FULL.pdf', 'rb') #Create pdf object
pdfReader = PyPDF2.PdfReader(pdfFileObj) 

In [2]:
allText = ""
for i in range(792): #Iterating over n pages 
    pageObj = pdfReader.pages[i]
    page = pageObj.extract_text()
    allText += page #Store page content into string variable

allText;

#### Split Text into persons

---

In [3]:
# Preprocessing
allText = allText.replace(';','')

alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information').replace('Remark',';Remark')
testListSplit = alltext_string_all.split('reference number: EU')
alltext_string_all[:1000]


'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\n;Legal basis: 2017/404 (OJ L63)\n;Programme: AFG - Afghanistan\n;Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Nam

In [4]:
# Create dataframe
dbn_df = pd.DataFrame([sub.split(";") for sub in testListSplit])
dbn_df.fillna("",inplace=True)
dbn_df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information', 'Remark','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']



In [5]:
dbn_df

,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,...,,,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,...,,,,,,,,,,
1,.1787.1\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Sirajuddi...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,...,,,,,,,,,,
2,.2272.60\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Nasiruddi...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n,Remark: A leader of the Haqqani Network (TAe.0...,,...,,,,,,,,,,
3,.2274.87\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Gul Agha ...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,...,,,,,,,,,,
4,.2291.0\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Amir Abdu...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,.9027.6\n,Legal basis: 2022/2229 (OJ L293-I)\n,Programme: UKR - Ukraine\nCompany name:\n• Nam...,,,,,,,,...,,,,,,,,,,
3636,.9028.5\n,Legal basis: 2022/2229 (OJ L293-I)\n,Programme: UKR - Ukraine\nCompany name:\n• Nam...,,,,,,,,...,,,,,,,,,,
3637,.5121.69\n,Legal basis: UNLI - 14.05.2019\n,Programme: UNLI - UN Listing\nPage 790 on 792E...,Remark: Islamic State of Iraq and the Levant -...,,,,,,,...,,,,,,,,,,
3638,.7817.36\n,Legal basis: 2022/419 (OJ L86)\n,Programme: YEM - Yemen\nCompany name:\n• Name:...,,,,,,,,...,,,,,,,,,,


## Data Cleanup 
---

In [6]:
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('\n','').replace('Name/Alias:',';').replace('•','').replace('Function:',''))
    dataframe.replace('^\s+', '', regex=True, inplace=True) #front
    dataframe.replace('\s+$', '', regex=True, inplace=True) #end
    dataframe['Birth information'] = dataframe['Birth information'].apply(lambda x: x.replace('Birth information:\n•',''))
    

    return dataframe

def stripDataframe(dataframe):
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.strip())
    
    return dataframe

def listCreation(dataframe):
    identityInformation = []
    for i in dataframe:
        temp = dataframe.tolist()
        identityInformation.append(temp)
    return identityInformation


In [7]:
cleanupDataframe(dbn_df)
stripDataframe(dbn_df)


,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,...,,,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,...,,,,,,,,,,
1,.1787.1,2017/404 (OJ L63),AFG - Afghanistan,; Sirajuddin Jallaloudine Haqqani Na’ib Amir...,Birth date: Circa from 1977 to 1978 Birth pl...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,...,,,,,,,,,,
2,.2272.60,2017/404 (OJ L63),AFG - Afghanistan,; Nasiruddin Haqqani A leader of the Haqqani...,Birth date: Circa from 1970 to 1973 Birth pl...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan,Remark: A leader of the Haqqani Network (TAe.0...,,...,,,,,,,,,,
3,.2274.87,2017/404 (OJ L63),AFG - Afghanistan,; Gul Agha Ishakzai ; Mullah Gul Agha ; Mu...,Birth date: Circa 1972 Birth place: Afghanis...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,...,,,,,,,,,,
4,.2291.0,2017/404 (OJ L63),AFG - Afghanistan,; Amir Abdullah Former Kandahar Province Dep...,Birth date: Circa 1972 Birth place: Afghanis...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,.9027.6,2022/2229 (OJ L293-I),UKR - UkraineCompany name:• Name: Islāma revol...,,,,,,,,...,,,,,,,,,,
3636,.9028.5,2022/2229 (OJ L293-I),UKR - UkraineCompany name:• Name: صنایع هوایی ...,,,,,,,,...,,,,,,,,,,
3637,.5121.69,UNLI - 14.05.2019,UNLI - UN ListingPage 790 on 792European Union...,Remark: Islamic State of Iraq and the Levant -...,,,,,,,...,,,,,,,,,,
3638,.7817.36,2022/419 (OJ L86),YEM - YemenCompany name:• Name: ANSARALLAH• Na...,,,,,,,,...,,,,,,,,,,


In [14]:
dbn_df["Identity information"][1]

'; Sirajuddin Jallaloudine Haqqani   Na’ib Amir (Deputy Commander)   ; Khalifa   ; Saraj Haqani   ; Siraj Haqani   ; Serajuddin Haqani   ; Siraj Haqqani'

In [9]:
print(dbn_df.loc[10])

EU Reference                                                         .296.86
Legal basis                                                2017/404 (OJ L63)
Programme                                                  AFG - Afghanistan
Identity information       ; Ezatullah Haqqani Khan Sayyid  Title: Maulav...
Birth information           Birth date: Circa 1957  Birth place: Afghanis...
Citizenship information    Citizenship information:\n• Citizenship: Afgha...
Contact information        Remark: Believed to be in Afghanistan/Pakistan...
Remark                                                                      
                                                                            
                                                                            
                                                                            
                                                                            
                                                                            

### Split into Persons Dataframe and Company Dataframe

In [10]:
print(dbn_df.columns.tolist())

['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information', 'Remark', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


## Fuzzy string matching
---

In [15]:
from thefuzz import fuzz, process 

# Preprocessing
n = dbn_df['Identity information'].to_string().split()
n

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


['0',
 '1',
 ';',
 'Sirajuddin',
 'Jallaloudine',
 'Haqqani',
 'Na’ib',
 'Amir...',
 '2',
 ';',
 'Nasiruddin',
 'Haqqani',
 'A',
 'leader',
 'of',
 'the',
 'Haqqani...',
 '3',
 ';',
 'Gul',
 'Agha',
 'Ishakzai',
 ';',
 'Mullah',
 'Gul',
 'Agha',
 ';',
 'Mu...',
 '4',
 ';',
 'Amir',
 'Abdullah',
 'Former',
 'Kandahar',
 'Province',
 'Dep...',
 '5',
 ';',
 'Agha',
 'Jan',
 'Alizai',
 ';',
 'Haji',
 'Agha',
 'JanAlizai',
 ';',
 '...',
 '6',
 ';',
 'Khalil',
 'Ahmed',
 'Haqqani',
 'Title:',
 'Haji',
 ';',
 'Khalee...',
 '7',
 ';',
 'Saleh',
 'Mohammad',
 'Kakar',
 'Akhtar',
 'Muhammad',
 ';',
 'Sal...',
 '8',
 ';',
 'Hamidullah',
 'Akhund',
 'Title:',
 'Mullah',
 'Head',
 'of',
 'A...',
 '9',
 ';',
 'Pahlawan',
 'Shamsuddin',
 ';',
 'Shamsuddin',
 'Title:',
 '(...',
 '10',
 ';',
 'Ezatullah',
 'Haqqani',
 'Khan',
 'Sayyid',
 'Title:',
 'Maulav...',
 '11',
 ';',
 'Aminullah',
 'Amin',
 'Quddus',
 'Title:',
 'Maulavi',
 'Gove...',
 '12',
 ';',
 'Mullah',
 'Jabar',
 ';',
 'Muawin',
 'Jabbar',

In [59]:
#name = 'Pascal Breucker'


# Search function
def findBusinessPartner(input, name, threshold):
    namelist = name.split(" ")
    print(namelist)
    count = 0
    resultlist = []
    for name in namelist:

        for i in input:
        
            if fuzz.ratio(i, name) >= threshold:
                count += 1
                #print(fuzz.ratio(i, name), name, i)
                resultlist.append([fuzz.ratio(i,name) ,name,i])
                
    resultlist.sort(key = lambda x: x[0], reverse = True)
    if count == 0: 
        print(f'Your requested Business partner {name} with fuzzy threshold {threshold} cannot be found on the EU Sanctions List')
    
    return resultlist


# to-do: fuzz.ratio selbst coden 

In [60]:
#name = 'siray Qusay'
name = input('gib dein scheiß hier ein')
findBusinessPartner(n, name, 60)

['qusay']


[[80, 'qusay', 'Qusay'],
 [80, 'qusay', 'Qusay'],
 [73, 'qusay', 'Husayn'],
 [73, 'qusay', 'Husayn'],
 [73, 'qusay', 'Husayn'],
 [67, 'qusay', 'Musa'],
 [67, 'qusay', 'Musa'],
 [67, 'qusay', 'Musa'],
 [67, 'qusay', 'Musa'],
 [67, 'qusay', 'Aqsa'],
 [67, 'qusay', 'Musa'],
 [67, 'qusay', 'Musa'],
 [62, 'qusay', 'Husay...'],
 [60, 'qusay', 'Musab'],
 [60, 'qusay', 'Musab'],
 [60, 'qusay', 'Mousa'],
 [60, 'qusay', 'Husam'],
 [60, 'qusay', 'Musab']]

## Word2Vec
---

In [ ]:
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib import pyplot
 
# #Preprocessing
# data = dbn_df['Identity information'] = dbn_df['Identity information'].apply(lambda content: [x for x in content.split(';')]) #Translating 'Identity information' into list object

# sentences_tokenized = []
# for i in data:
#     print(i)
#     # s = i.lower
#     # myPunc = '!"#%&$()*+-./:;,<=>?@[\\]^_`{|}~'
#     # s = s.translate(s.maketrans(myPunc, ' '*len(myPunc)))
#     # s = s.translate(s.maketrans(string.digits, ' '*len(string.digits)))
#     # s = re.sub('\s+',' ',s)
#     # print(s)
#     # for sentence in s:    
#     #     tokens = sentence.split(' ')
#     #     sentences_tokenized.append(tokens)




In [ ]:
# data1 = []
# for i in dbn_df['Identity information']:
#     # temp = []
#     # for j in i:
#     #     temp.append(j.lower())
#     # data.append(temp)
#     data1.append(i.lower())


ImportError: cannot import name 'word2vec' from 'gensim' (c:\Users\DELSTEI9\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\__init__.py)

In [ ]:
# Creating Word2Vec
model = Word2Vec(
    sentences = n,
    #size = 50,
    window = 10,
    #iter = 20,
    
)




In [ ]:
model.wv.key_to_index #Ausgabe aller Key-Vektor Paare in Word2Vec model


{'a': 0,
 '.': 1,
 'i': 2,
 'l': 3,
 'e': 4,
 ';': 5,
 'r': 6,
 'A': 7,
 'h': 8,
 'n': 9,
 't': 10,
 'd': 11,
 'o': 12,
 'u': 13,
 'm': 14,
 'M': 15,
 'v': 16,
 'j': 17,
 'S': 18,
 's': 19,
 'T': 20,
 '2': 21,
 '1': 22,
 'I': 23,
 'а': 24,
 '3': 25,
 'K': 26,
 'k': 27,
 'V': 28,
 'H': 29,
 'N': 30,
 'е': 31,
 'b': 32,
 'J': 33,
 'А': 34,
 'и': 35,
 'O': 36,
 ':': 37,
 'R': 38,
 'c': 39,
 'р': 40,
 'л': 41,
 'н': 42,
 'в': 43,
 'U': 44,
 'L': 45,
 'E': 46,
 'g': 47,
 'D': 48,
 'f': 49,
 '4': 50,
 '7': 51,
 '6': 52,
 '5': 53,
 'y': 54,
 '8': 55,
 '0': 56,
 '9': 57,
 'о': 58,
 'z': 59,
 'B': 60,
 'G': 61,
 'ч': 62,
 'Y': 63,
 'В': 64,
 'q': 65,
 'C': 66,
 'P': 67,
 'і': 68,
 'д': 69,
 'к': 70,
 'К': 71,
 'с': 72,
 'О': 73,
 'т': 74,
 'й': 75,
 'Н': 76,
 'И': 77,
 'F': 78,
 'С': 79,
 'я': 80,
 'М': 81,
 'p': 82,
 'ь': 83,
 'Z': 84,
 'м': 85,
 'Р': 86,
 '(': 87,
 'г': 88,
 'Е': 89,
 'x': 90,
 'У': 91,
 ')': 92,
 'Л': 93,
 'w': 94,
 'Д': 95,
 'Q': 96,
 'Т': 97,
 'Г': 98,
 ',': 99,
 'Б': 100,

In [ ]:
ergebnisse = model.wv.most_similar(name, topn=5)

for y in ergebnisse:
    print(y)

KeyError: "Key 'Sirajuddin' not present in vocabulary"